# Automated Transient Determination

In [1]:
from __future__ import division, print_function, unicode_literals
import os
import os.path
import glob
import itertools
import subprocess
import ccdproc
from astropy import units as u
import numpy as np
import astropy.io.fits as fits
import sys
import shutil as shutil
from shutil import move
import pyfits
from astropy.convolution import Gaussian2DKernel, convolve
import ephem

### Here we can add the below process to the existing Transients code

In [2]:
#This section tells the code where the files are. 
#data_path should point to the directory that holds the .resamp.fits files 
#produced by swarp (usually by Jack's Code)
data_path = ('./output/Temp/')

#master_file is the final processed image made from the above resamp files.
master_file = ['./output/coadd_ngc300_r.fits']

#master_weight is the weight file which corresponds to the above master file.
master_weight = ['./output/coadd_ngc300_r.weight.fits']

In [3]:
#These variables are lists of the resampled files and their corresponding weights.

resamps=sorted(glob.glob(data_path+'*.resamp.fits'))
weights=sorted(glob.glob(data_path+'*.resamp.weight.fits'))

In [10]:
#Creates a segmentation map from the master file. This will be used to generate flag files later.
#To tweak the sensitivity of this segmentation map (brightness of objects it identifies for flagging), change
#the DETECT_THRESH and ANALYSIS_THRESH values in segmentation.sex, located in the config_files directory.
#This file is a temporary file, but the code will not delete it by default, as it can be a useful analysis tool.

subprocess.call('sex '+master_file[0]+' -c ./config_files/segmentation.sex -WEIGHT_IMAGE '+master_weight[0], shell=True)
subprocess.call('mv ./Processed_Images/Temp/check.fits ./Processed_Images/Temp/segmentation_map_ref.fits', shell=True)
master_segmentation = ['./Processed_Images/Temp/segmentation_map_ref.fits']

In [12]:
#This loop creates all the difference images and their respective object, background, and aperture maps.

for i in range(0,2):#len(resamps)):
    reference=fits.open(master_file[0])
    resampled=fits.open(resamps[i])
    
    #measures the distance from the central WCS pixel to the right and to the bottom of the image, respectively.
    resdistx=(resampled[0].header['NAXIS1']-resampled[0].header['CRPIX1'])
    resdisty=(resampled[0].header['NAXIS2']-resampled[0].header['CRPIX2'])
    
    #Creates difference values that represent the start and end pixels of the resampled image on the reference image.
    ref_x_start=int(reference[0].header['CRPIX1']-resampled[0].header['CRPIX1'])
    ref_y_start=int(reference[0].header['CRPIX2']-resampled[0].header['CRPIX2'])
    ref_x_end=int(reference[0].header['CRPIX1']+resdistx)
    ref_y_end=int(reference[0].header['CRPIX2']+resdisty)
    
    #loading files into memory
    refimg=pyfits.open(master_file[0])
    resimg=pyfits.open(resamps[i])
    segimg=pyfits.open(master_segmentation[0])
    
    #converting to array
    D1=refimg[0].data
    D2=resimg[0].data
    D3=segimg[0].data
    
    #resizing and scaling pixel values
    resize_ref=D1[ref_y_start:ref_y_end,ref_x_start:ref_x_end]
    resamp_scaled=D2*resampled[0].header['FLXSCALE']
    resize_seg=D3[ref_y_start:ref_y_end,ref_x_start:ref_x_end]
    
    #doing the subtraction of the reference image from the resampled
    out_file=resamp_scaled-resize_ref
    
    #loading header information from original resampled image. This will be saved into the new images.
    head=pyfits.getheader(resamps[i])
    
    #loads the filename of the resamp image into a variable, then deletes .resamp.fits from the end of it.
    #This allows the code to write new files with the same naming format, but different file extension names.
    #If your files don't end in .resamp.fits, the -12 will need to be changed to match the number of characters
    #in the file extension.
    path=os.path.basename(resamps[i])
    new_path=path[:-12]
    
    #saves the final difference file, and the temporary resized segmentation map.
    pyfits.writeto(('./Processed_Images/'+new_path+'.difference.fits'), out_file, head)
    pyfits.writeto(('./Processed_Images/Temp/temp_segment.fits'), resize_seg, head)
    
    #produces a flag map from the temporary segmentation map.
    subprocess.call('ww -c ./config_files/default.ww -WEIGHT_NAMES '+weights[i]+',./Processed_Images/Temp/temp_segment.fits', shell=True)
    
    #does the main sextractor run on the subtracted file, producing the object, background, and aperture files.
    #this is controlled by default.sex in the config_files directory
    subprocess.call('sex ./Processed_Images/'+new_path+'.difference.fits -c ./config_files/default.sex', shell=True)
    
    #cleans up temporary files and renames background, object, and aperture files. 
    subprocess.call('rm ./Processed_Images/Temp/temp_segment.fits',shell=True)
    subprocess.call('mv ./Processed_Images/check.fits ./Processed_Images/'+new_path+'.background.fits',shell=True)
    subprocess.call('mv ./Processed_Images/check2.fits ./Processed_Images/'+new_path+'.object.fits',shell=True)
    subprocess.call('mv ./Processed_Images/check3.fits ./Processed_Images/'+new_path+'.apertures.fits',shell=True)
    subprocess.call('rm ./Processed_Images/Temp/weight.fits',shell=True)
    subprocess.call('rm ./Processed_Images/Temp/flag.fits',shell=True)
#Uncomment the next line if you would like the code to automatically clean up the master segmentation map.
#subprocess.call('rm ./Processed_Images/Temp/segmentation_map_ref.fits',shell=True)

0

# Begin Brendan's Transient checking script:

In [ ]:
# this is where the code can be tacked onto the pipeline, after subtracting reference images.
# At this point we should have a combined image, with reference subtracted, where the only point sources remaining
# (in theory) are transients such as supernovae or asteroids.

# This code aims to identify such sources as either asteroids or anomalies, which can then be manually followed up.

### Run SExtractor once more to spot remaining (transient) sources:

Generate list of sources:

## Check against catalogues of known asteroids:

In [2]:
# Download latest MPCORB catalogue from:
# www.minorplanetcenter.org/iau/MPCORB/MPCORB.DAT

# Full list of paramters and how to read the data:
# http://www.minorplanetcenter.net/iau/info/MPOrbitFormat.html


# and do a manual check using PYEPHEM.

In [78]:
# Load MPCORB.DAT data into an array:
# delimiter to split data into required columns
# Add 'skip_header=41' parameter when using real data to skip header info
# names = ['designation', 'abs mag', 'slope param', 'epoch', 'mean epoch anomaly', 'perihelion arg', 'longitude', 'inclination', 'eccentricity', 'mean daily motion', 'semimajor axis', 'uncertainty', 'reference', 'num obs', 'num opp', 'obs years / arc length', 'rms', 'coarse perturb', 'precise perturb', 'comp name', '4hex flag', 'readable designation',
# 'last obs']
MPCORB = np.genfromtxt('MPCORB/MPCORB_test', autostrip=True, dtype=str,  delimiter = [8,6,6,6,10,11,11,11,11,12,12,3,10,6,4,10,5,4,4,11,5,28,8])
MPCORB[0]

array(['00004', '3.20', '0.32', 'K161D', '129.52980', '151.13748',
       '103.84611', '7.14005', '0.0889223', '0.27156600', '2.3616695', '0',
       'MPO358623', '6811', '98', '1821-2015', '0.60', 'M-p', '18h',
       'MPCLINUX', '0000', '(4) Vesta', '20151215'], 
      dtype='|S11')

### Search MPCORB for matches:

In [ ]:
# Nest this loop outside the MPCORB search loop:
# for trans in translist:
    # RA = trans.header["RA"]
    # DEC = trans.header["DEC"]
    # obstime = trans.header["DATE"]
    # identcount = 0
    # for object in MPCORB:
        # .......
    
    # if identcount = 0 (no object found):
        # anom.append("object designation", "object name" "trans WCS at that time", )

Required orbital params:

N (= Om) = longitude of the ascending node 

i (= inc) = inclination to the ecliptic (plane of the Earth's orbit)

w (= om) = argument of perihelion

a = semi-major axis, or mean distance from Sun

e = eccentricity (0=circle, 0-1=ellipse, 1=parabola)

M = mean anomaly (0 at perihelion; increases uniformly with time)

## Fiddling around with pyephem:

### Define a function to extract the packed date format given in the MPCORB data:

In [210]:
# define dictionary of alpha to numbers for MPCORB packed dates translation:
# http://www.minorplanetcenter.net/iau/info/PackedDates.html

def dateextract(date):
    packeddates = {'1':'1', '2':'2', '3':'3', '4':'4', '5':'5', '6':'6', '7':'7', '8':'8', '9':'9', \
                   '0':'0', 'A':'10', 'B':'11', 'C':'12', 'D':'13', 'E':'14', 'F':'15', 'G':'16', \
                   'H':'17', 'I':'18', 'J':'19', 'K':'20', 'L':'21', 'M':'22', 'N':'23', 'O':'24', \
                   'P':'25', 'Q':'26', 'R':'27', 'S':'28','T':'29', 'U':'30', 'V':'31' \
              }
    datestring = list(date)
    datestring2 = ""
    for i in range(len(datestring)):
        char = packeddates['%s' % datestring[i]]
        datestring[i] = char
        datestring2 += str(datestring[i])
    # now conjoin array back to string, as a date format required by ephem.. eg 
    return datestring2
    
    print(datestring2)

In [211]:
dateextract('K161D')

u'2016113'

### Load some asteroid parameters into pyephem to test:

In [106]:
# initialise a new, blank orbital body and load orbital params:
params = MPCORB[0]
body = ephem.EllipticalBody()

#six required params for Keplerian orbit:
body._Om = float(params[6])
body._inc = float(params[7])
body._om = float(params[5])
body._a = float(params[10])
body._e = float(params[8])
body._M = float(params[4])

#and mean daily motion n required too:
mdm = aasdasdsads

# Correction for precession:
N = N_Epoch + 0.013967 * ( 2000.0 - Epoch ) + 3.82394E-5 * d


print(params)
print(body._Om, body._inc, body._om, body._a, body._e, body._M)

['00004' '3.20' '0.32' 'K161D' '129.52980' '151.13748' '103.84611'
 '7.14005' '0.0889223' '0.27156600' '2.3616695' '0' 'MPO358623' '6811' '98'
 '1821-2015' '0.60' 'M-p' '18h' 'MPCLINUX' '0000' '(4) Vesta' '20151215']
103:50:46.0 7:08:24.2 151:08:14.9 2.36166954041 0.0889223 129:31:47.3


In [ ]:
# correction for precession
N = params[6]

# NEEDTO FIND A WAY TO CONVERT THE PACKED EPOCH FORM TO A DATE TO MAKE THIS CORRECTION UNIVERSAL

In [107]:
# using orbital params, predict location in sky of object at certain time:


In [ ]:
For MP in MPCORB:
    inc = 
     = 
    newobject = []
    
    
    
    # convert orbital params at given date to RA/DEC
    # Vec = vectorize (trans RA & DEC) with (MPCORB object RA & DEC)
    # if Vec <= threshold (20 arcmin?):
        # ident.append("object designation", "object name" "object WCS at that time", "distance vector from trans")
        # identcount += 1
   

In [201]:
m = ephem.Sun()
m.compute("2004/4/20")
m.ra, m.dec

ValueError: dates must be initialized from a number, string, tuple, or datetime

In [203]:
print(ephem.Date("2004/3/27"))

ValueError: dates must be initialized from a number, string, tuple, or datetime

In [180]:
print(ephem.Date(0.3875488833))

1899/12/31 21:18:04


### Return anomalies:

In [ ]:
# Use pyephem, take WCS of anomalies, if pyephem of each oject in MPCORB is within a threshold (vectorize) at that time, then return object
# else, return anomaly

In [52]:
print(len('00001    3.34  0.12 K161D 181.38133   72.73324   80.32180   10.59166  0.0757544  0.21400734   2.7681117  0 MPO358623  6592 109 1801-2015 0.60 M-v 30h MPCLINUX   0000      (1) Ceres              20151128'))

202
